<a href="https://colab.research.google.com/github/rubuntu/Taller_Introduccion_a_Ciencia_de_Datos_IA_e_Ingenieria_de_Datos/blob/main/sesion_16_fundamentos_de_rag_con_hugging_face_chromadb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📘 Sesión 16 – Fundamentos de RAG con Hugging Face + ChromaDB

## Objetivos

* Comprender los conceptos básicos de RAG y su arquitectura.
* Aprender a transformar texto → embeddings → almacenar en base vectorial.
* Construir un mini-RAG con ChromaDB y Hugging Face embeddings.

---

## Contenido

1. **Concepto de RAG**:

   * Separar *memoria a largo plazo* (vector store) del LLM.
   * Flujo: **Pregunta → Embedding → Recuperación → Contexto → LLM → Respuesta**.

2. **Primer prototipo**:

   * Usar `sentence-transformers` de Hugging Face para embeddings.
   * Guardar y consultar embeddings en **ChromaDB**.
   * Pasar contexto recuperado a un LLM (ej. `transformers` o `openai`).

---

## Demo Código

In [1]:
# ========================
# Instalación de librerías
# ========================
%%capture
!pip install -q -U wikipedia-api chromadb bitsandbytes gradio


In [2]:
# ========================
# 1. Cargar datos de Wikipedia (Paraguay)
# ========================
import wikipediaapi

wiki_wiki = wikipediaapi.Wikipedia(
    language='es',
    user_agent='MiProyectoRAG/1.0 (https://github.com/rubuntu)'
)

paginas = [
    "Paraguay",
    "Geografía de Paraguay",
    "Economía de Paraguay",
    "Cultura de Paraguay",
    "Historia de Paraguay",
    "Asunción",
     #"Guerra de la Triple Alianza", "Guerra del Chaco"
]

docs = []
for titulo in paginas:
    page = wiki_wiki.page(titulo)
    if page.exists():
        docs.append(page.text)
        print(f"Cargado: {titulo}")
print(f"Total documentos cargados: {len(docs)}")


Cargado: Paraguay
Cargado: Historia de Paraguay
Cargado: Geografía de Paraguay
Cargado: Economía de Paraguay
Cargado: Cultura de Paraguay
Cargado: Asunción
Cargado: Guerra de la Triple Alianza
Cargado: Guerra del Chaco
Total documentos cargados: 8


In [3]:
# ========================
# 2. Chunking para RAG
# ========================
def chunk_text(text, max_chars=1200, overlap=200):
    chunks, start = [], 0
    while start < len(text):
        end = start + max_chars
        chunk = text[start:end]
        chunks.append(chunk)
        start += max_chars - overlap
    return chunks

all_chunks = []
for doc in docs:
    all_chunks.extend(chunk_text(doc))

print(f"Total chunks creados: {len(all_chunks)}")


Total chunks creados: 613


In [4]:
# ========================
# 3. Embeddings y ChromaDB
# ========================
from sentence_transformers import SentenceTransformer
import chromadb

emb_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = emb_model.encode(all_chunks, batch_size=16, show_progress_bar=True)

client = chromadb.Client()
collection = client.create_collection("paraguay_wiki")

for i, chunk in enumerate(all_chunks):
    collection.add(documents=[chunk], embeddings=[embeddings[i].tolist()], ids=[str(i)])

print("✅ Base vectorial creada con ChromaDB.")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/39 [00:00<?, ?it/s]

✅ Base vectorial creada con ChromaDB.


In [5]:
# ========================
# 4. Cargar Modelo
# ========================
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "unsloth/gemma-2b-it-bnb-4bit"  # versión ligera y estable
model_id = "unsloth/gemma-3n-E2B-it-unsloth-bnb-4bit"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True, use_fast=False)
llm = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=True
)

print("✅ Modelo cargado:", model_id)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.70M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/469M [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

✅ Modelo cargado: unsloth/gemma-3n-E2B-it-unsloth-bnb-4bit


In [20]:
# ========================
# 5. Función RAG
# ========================
def to_gemma_chat(user_text):
    return f"<bos><start_of_turn>user\n{user_text}<end_of_turn>\n<start_of_turn>model\n"

def rag_answer(question, top_k=3, max_new_tokens=200, show_context=False):
    # Recuperar contexto
    query_emb = emb_model.encode([question])
    results = collection.query(query_embeddings=query_emb.tolist(), n_results=top_k)
    context = "\n".join(results["documents"][0])

    if show_context:
        print("\n--- CONTEXTO ---")
        print(context[:1000] + ("..." if len(context) > 1000 else ""))
        print("--- FIN CONTEXTO ---\n")

    # Prompt
    prompt = f"""
Responde a la pregunta basándote únicamente en el siguiente contexto.
Si el contexto no contiene la respuesta, responde "No lo sé".

Contexto:
{context}

Pregunta: {question}

Respuesta:
"""
    chat_prompt = to_gemma_chat(prompt)

    inputs = tokenizer(chat_prompt, return_tensors="pt", truncation=True, max_length=4096).to("cuda")
    outputs = llm.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.3,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # --- limpieza de la salida ---
    # cortar donde empieza <start_of_turn>model
    if "<start_of_turn>model" in decoded:
        answer = decoded.split("<start_of_turn>model")[-1].strip()
    else:
        # fallback: quitar el prompt entero si aparece
        answer = decoded.replace(chat_prompt, "").strip()

    return answer


In [23]:
# ========================
# 6. Preguntas de ejemplo
# ========================
questions = [
    #"¿Cuál es la capital de Paraguay?",
    #"¿Qué importancia tiene el río Paraguay?",
    "Comenta sobre el río Paraguay",
    #"¿Quién fue Francisco Solano López?",
    #"¿Qué papel tuvo Paraguay en la Guerra de la Triple Alianza?",
    #"¿Cuáles son los principales productos de exportación de Paraguay?"
]

for q in questions:
    print("\n❓", q)
    print("💡", rag_answer(q, ))


❓ Comenta sobre el río Paraguay
💡 El contexto menciona que Paraguay reivindicaba la frontera del río Yaurú con el territorio paraguayo. Sin embargo, también se menciona que Paraguay pudo considerarse aliado del Brasil por dos razones: por un lado, se habían abierto dos zonas francas para comerciar con el Brasil, en Itapúa y en Fuerte Olimpo; por otro lado, la negativa del gobernador Juan Manuel de Rosas a reconocer la independencia paraguaya y su derecho a comerciar a través del río Paraná llevó a una alianza tácita entre las dos naciones. Esto sugiere que el río Paraguay fue importante para el comercio y las relaciones entre Paraguay y Brasil en ese período. El contexto no especifica detalles sobre el propio río Paraguay, sino que se centra en la reivindicación de su frontera por parte de Paraguay.


In [8]:
## ========================
## 7. Interfaz con Gradio
## ========================
#import gradio as gr

#def ask_question(query):
#    return rag_answer(query)

#demo = gr.Interface(
#    fn=ask_question,
#    inputs=gr.Textbox(lines=2, placeholder="Escribe tu pregunta sobre Paraguay..."),
#    outputs="text",
#    title="RAG sobre Paraguay (Wikipedia + LLM)",
#    description="Haz preguntas sobre Paraguay usando RAG con ChromaDB y un modelo abierto (Gemma, Mistral, Llama, Qwen)."
#)

#demo.launch(share=True)

---

## Preguntas de discusión

1. ¿Por qué un LLM necesita un vector store externo para RAG?
2. ¿Qué limitaciones tendría un mini-RAG con pocas docenas de documentos?
3. ¿Cómo escalarías esto a millones de documentos?

---